# Conversation Patterns & Context

Building on:
- **001**: `chat()` vs `communicate()`
- **002**: Multi-agent collaboration 
- **003**: `operate()` with function calling

**This notebook**: Practical conversation patterns and context management.

In [1]:
from lionagi import Branch, iModel

## Pattern 1: Customer Service with Context

Using `communicate()` to maintain conversation state:

In [2]:
# Simple password reset function
def reset_password(user_id: str) -> str:
    """Reset user password and return new temporary password."""
    return f"Password reset for {user_id}. Temporary password: temp123"


# Customer service agent
agent = Branch(
    system="You are a helpful customer service agent. Be concise and friendly.",
    tools=[reset_password],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

In [3]:
# Multi-turn conversation with context
user_id = "alice123"

# First message
response1 = await agent.communicate(
    "Hi, I can't log into my account", context={"user_id": user_id}
)
print("Agent:", response1)

# Follow-up (context is maintained)
response2 = await agent.communicate("I think I need a password reset")
print("\nAgent:", response2)

# Check if any tools were used
if hasattr(agent, "last_response") and agent.last_response:
    print(
        f"\nTools used: {len([msg for msg in agent.messages if msg.role == 'action'])}"
    )

Agent: Hi Alice! Sorry to hear you're having trouble logging in. Could you please let me know if you're seeing any error messages? Meanwhile, you might try resetting your password using the "Forgot Password" link. Let me know if you need further help!

Agent: Hi Alice! I’ve sent a password reset link to your registered email. Please check your inbox and follow the instructions. Let me know if you don’t receive it!


## Pattern 2: Conversation with Memory

Building context across multiple exchanges:

In [4]:
# Shopping assistant that remembers preferences
shopper = Branch(
    system="You are a shopping assistant. Remember user preferences and make personalized recommendations."
)

# Build up preferences over time
pref1 = await shopper.communicate("I like blue colors and minimalist design")
print("Assistant:", pref1)

pref2 = await shopper.communicate("My budget is around $100-200")
print("\nAssistant:", pref2)

# Now ask for recommendations (uses accumulated context)
rec = await shopper.communicate("What laptop should I buy?")
print("\nRecommendation:", rec)

# Check conversation length
print(f"\nConversation has {len(shopper.messages)} messages")

Assistant: Thank you for sharing your preferences! Based on your love for blue colors and minimalist design, I recommend exploring the following options:

1. **Blue Minimalist Smartphone Case** – Sleek, solid blue, and simple in design, perfect for a modern look.
2. **Minimalist Blue Watch** – A clean, uncluttered face with a simple blue strap for everyday elegance.
3. **Blue Ceramic Vase** – Elegant, with smooth lines and a minimalist aesthetic that complements any space.
4. **Blue Leather Wallet** – Streamlined design with a vibrant blue hue, ideal for a minimalist style.
5. **Blue Abstract Art Print** – Simple yet striking artwork featuring shades of blue, perfect for minimalist decor.

Would you like me to find specific brands or shops for any of these items?

Assistant: Great! With a budget of $100-200, you can find high-quality minimalist blue items within your price range. Here are some tailored recommendations:

1. **Blue Minimalist Smartwatch** – Brands like Garmin or Withings

## Pattern 3: Conversation State Management

Viewing and managing conversation history:

In [5]:
# View conversation as DataFrame
conversation_df = shopper.to_df()
print("Conversation overview:")
print(conversation_df[["role", "content"]].head())

# Get just user messages
user_messages = [msg for msg in shopper.messages if msg.role == "user"]
print(f"\nUser said {len(user_messages)} things")

# Get just assistant responses
assistant_messages = [
    msg for msg in shopper.messages if msg.role == "assistant"
]
print(f"Assistant responded {len(assistant_messages)} times")

Conversation overview:
        role                                            content
0     system  {'system_message': 'You are a shopping assista...
1       user  {'context': [], 'instruction': 'I like blue co...
2  assistant  {'assistant_response': 'Thank you for sharing ...
3       user  {'context': [], 'instruction': 'My budget is a...
4  assistant  {'assistant_response': 'Great! With a budget o...

User said 3 things
Assistant responded 3 times


## Pattern 4: Context-Aware Operations

Combining `operate()` with conversation context:

In [6]:
def search_products(category: str, max_price: float) -> str:
    """Search for products in category under max price."""
    return f"Found 5 {category} items under ${max_price}"


def add_to_cart(product_name: str) -> str:
    """Add product to shopping cart."""
    return f"Added {product_name} to cart"


# Shopping bot with tools
shop_bot = Branch(
    system="You help users shop. Use tools when needed.",
    tools=[search_products, add_to_cart],
)

In [7]:
# Set shopping context
await shop_bot.communicate("I'm looking for wireless headphones under $150")

# Use operate() with accumulated context
result = await shop_bot.operate(
    instruction="Find me some options and add the best one to my cart",
    actions=True,
)

print("\nActions taken:")
for action in result.action_responses:
    print(f"- {action.function}: {action.output}")


Actions taken:
- search_products: Found 5 wireless headphones items under $150


## Key Patterns Summary

1. **Contextual Communication**: Use `context` parameter to provide relevant information
2. **Memory Building**: `communicate()` automatically maintains conversation history
3. **State Inspection**: Access `branch.messages` and `branch.to_df()` to understand conversation
4. **Context + Actions**: Combine conversation memory with `operate()` for context-aware tool usage

These patterns form the foundation for building conversational AI applications!